In [1]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier

### IDEA
La idea de este notebook es probar solo con los datos relacionados al area.

In [2]:
# En base a lo visto por el TP1 las siguientes areas son las mas
# postuladas
areas_importantes = [
    'Ventas', 'Administración', 'Producción', 'Comercial', 'Atención al Cliente',
    'Recepcionista', 'Call Center', 'Telemarketing', 'Tesorería', 'Mantenimient y Limpieza',
]

# 1. AVISOS

In [3]:
avisos = pd.concat([
    pd.read_csv("../input/datos/fiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/MISfiuba_6_avisos_detalle_missing_nivel_laboral.csv")
])
avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [4]:
# Solo me quedo con nombre de area
avisos = avisos[['idaviso', 'nombre_area']]
avisos.head()

,idaviso,nombre_area
0,8725750,Comercial
1,17903700,Salud
2,1000150677,Transporte
3,1000610287,Transporte
4,1000872556,Producción


In [5]:
avisos.drop_duplicates(subset='idaviso', inplace=True)
avisos['nombre_area'] = avisos['nombre_area'].apply(lambda x: x if x in areas_importantes else 'Otra Area')

In [6]:
# Uso one hot encoding para area
dummies_area = pd.get_dummies(avisos['nombre_area'])
avisos = pd.concat([avisos['idaviso'], dummies_area], axis=1)
avisos.head()

,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,8725750,0,0,0,1,0,0,0,0,0,0
1,17903700,0,0,0,0,1,0,0,0,0,0
2,1000150677,0,0,0,0,1,0,0,0,0,0
3,1000610287,0,0,0,0,1,0,0,0,0,0
4,1000872556,0,0,0,0,0,1,0,0,0,0


In [7]:
avisos.head()

,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,8725750,0,0,0,1,0,0,0,0,0,0
1,17903700,0,0,0,0,1,0,0,0,0,0
2,1000150677,0,0,0,0,1,0,0,0,0,0
3,1000610287,0,0,0,0,1,0,0,0,0,0
4,1000872556,0,0,0,0,0,1,0,0,0,0


# 2. VISTAS

In [8]:
vistas = pd.concat([
    pd.read_csv("../input/datos/fiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_3_vistas.csv")
])
vistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [9]:
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas = vistas[['idaviso', 'idpostulante']]
vistas.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [10]:
# Aniado la informacion del area de cada aviso
avisos_vistos = pd.merge(vistas, avisos, on='idaviso', how='inner')
avisos_vistos.head()

,idaviso,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,1111780242,YjVJQ6Z,0,0,0,0,1,0,0,0,0,0
1,1111780242,dYjRpMD,0,0,0,0,1,0,0,0,0,0
2,1111780242,wVopPXl,0,0,0,0,1,0,0,0,0,0
3,1111780242,wVoOPoL,0,0,0,0,1,0,0,0,0,0
4,1111780242,KBdJJKd,0,0,0,0,1,0,0,0,0,0


In [11]:
avisos_vistos.drop(columns='idaviso',inplace=True)

In [12]:
# Cada columna representa la cantidad de avisos vistos por area para cada postulante
avisos_vistos = avisos_vistos.pivot_table(index='idpostulante', aggfunc='sum')
avisos_vistos.head()

,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
idpostulante,,,,,,,,,,
0005E,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
00Lkv,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
00dMd,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
01QAq,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
021OM,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Ningun usuario supera 2^16 cantidad de vistas por area
avisos_vistos = avisos_vistos.astype('uint16')

In [14]:
# le agrego la palabra 'vistas' adelante para que sea mas claro y no se confunda luego con el area postulada
nuevos_nombres = []
for area in avisos_vistos.columns:
    area = "vistas " + area
    nuevos_nombres.append(area)
avisos_vistos.columns = nuevos_nombres
avisos_vistos.head()

,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas
idpostulante,,,,,,,,,,
0005E,0,0,0,0,1,0,0,0,0,0
00Lkv,0,0,0,0,1,0,0,0,0,0
00dMd,0,0,0,0,1,0,0,0,0,0
01QAq,0,0,0,0,1,0,0,0,0,0
021OM,0,0,0,0,1,0,0,0,0,0


In [15]:
# Con esos datos tenemos representados a los usuarios segun sus areas favoritas segun la cantidad de vistas que le dedicaron al area

# 3. POSTULACIONES

In [16]:
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [17]:
postulaciones.drop(columns='fechapostulacion', inplace=True)
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [18]:
postulaciones.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)

In [19]:
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [20]:
# Busco los avisos y las areas de cada postulacion
postulaciones_detalle = pd.merge(postulaciones, avisos, on='idaviso', how='inner')

In [21]:
# 1 en el area de la categoria
postulaciones_detalle.head()

,idaviso,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,1112257047,NM5M,0,1,0,0,0,0,0,0,0,0
1,1112257047,1kJqGb,0,1,0,0,0,0,0,0,0,0
2,1112257047,eOE9Rr,0,1,0,0,0,0,0,0,0,0
3,1112257047,Zrx8Xz,0,1,0,0,0,0,0,0,0,0
4,1112257047,ZrKNQY,0,1,0,0,0,0,0,0,0,0


In [22]:
# Finalmente uno la informacion de los usuarios y sus postulaciones
datos = pd.merge(avisos_vistos, postulaciones_detalle, on='idpostulante', how='inner')
datos.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0z5Dmrd,0,0,0,0,4,0,0,0,0,0,1112384041,0,0,0,0,1,0,0,0,0,0
1,0z5Dmrd,0,0,0,0,4,0,0,0,0,0,1112420060,0,0,0,0,1,0,0,0,0,0
2,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112292169,0,0,0,0,1,0,0,0,0,0
3,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112309728,0,0,0,0,1,0,0,0,0,0
4,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112257171,0,0,0,0,1,0,0,0,0,0


In [23]:
# Todos estos avisos fueron postulaciones y por lo tanto se clasifican como postulados
# POSTULADO 1
# NO_POSTULADO 0
datos['postulado'] = 1
datos.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas,postulado
0,0z5Dmrd,0,0,0,0,4,0,0,0,0,0,1112384041,0,0,0,0,1,0,0,0,0,0,1
1,0z5Dmrd,0,0,0,0,4,0,0,0,0,0,1112420060,0,0,0,0,1,0,0,0,0,0,1
2,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112292169,0,0,0,0,1,0,0,0,0,0,1
3,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112309728,0,0,0,0,1,0,0,0,0,0,1
4,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112257171,0,0,0,0,1,0,0,0,0,0,1


In [24]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5117287 entries, 0 to 5117286
Data columns (total 23 columns):
idpostulante                  object
vistas Administración         uint16
vistas Atención al Cliente    uint16
vistas Call Center            uint16
vistas Comercial              uint16
vistas Otra Area              uint16
vistas Producción             uint16
vistas Recepcionista          uint16
vistas Telemarketing          uint16
vistas Tesorería              uint16
vistas Ventas                 uint16
idaviso                       int64
Administración                uint8
Atención al Cliente           uint8
Call Center                   uint8
Comercial                     uint8
Otra Area                     uint8
Producción                    uint8
Recepcionista                 uint8
Telemarketing                 uint8
Tesorería                     uint8
Ventas                        uint8
postulado                     int64
dtypes: int64(2), object(1), uint16(10), uint8(1

# 4. Generacion de NO_POSTULACIONES

In [25]:
# En realidad es la info que describe a los usuarios segun sus vistas.
# TODO cambiar el nombre de avisos_vistos a usuarios
usuarios = avisos_vistos
usuarios.reset_index(inplace=True)
usuarios.head()


,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas
0,0005E,0,0,0,0,1,0,0,0,0,0
1,00Lkv,0,0,0,0,1,0,0,0,0,0
2,00dMd,0,0,0,0,1,0,0,0,0,0
3,01QAq,0,0,0,0,1,0,0,0,0,0
4,021OM,0,0,0,0,1,0,0,0,0,0


In [26]:
usuarios.size

4402079

In [27]:
# Categoria de cada aviso
avisos.head()

,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,8725750,0,0,0,1,0,0,0,0,0,0
1,17903700,0,0,0,0,1,0,0,0,0,0
2,1000150677,0,0,0,0,1,0,0,0,0,0
3,1000610287,0,0,0,0,1,0,0,0,0,0
4,1000872556,0,0,0,0,0,1,0,0,0,0


In [28]:
# Mezclo al azar usuarios con sus vistas mas algun aviso cualquiera.
N = 5000000
usuarios_random = usuarios.sample(n=N, replace=True, random_state=44)
avisos_random = avisos.sample(n=N, replace=True, random_state=99)

In [29]:
usuarios_random.reset_index(inplace=True)
avisos_random.reset_index(inplace=True)
usuarios_random.drop(columns='index', inplace=True)
avisos_random.drop(columns='index', inplace=True)

In [30]:
avisos_random.size

55000000

In [31]:
# No se porque tienen tamanios distintos asi que solo tomo los primeros N de 
# avisos_random
datos_random = pd.concat([usuarios_random, avisos_random], axis=1)

In [32]:
datos_random.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,1Q4bd1R,1,0,1,0,2,0,1,0,0,0,1112397732,0,0,0,0,0,1,0,0,0,0
1,PmGkoMv,0,0,0,0,1,1,0,0,0,0,1112342096,0,0,0,0,1,0,0,0,0,0
2,ZDr5jaZ,0,0,0,0,1,0,0,0,0,0,1112225199,0,0,0,0,0,0,0,0,0,1
3,KBdV1oj,0,3,0,0,5,1,0,0,1,9,1112322633,0,0,0,0,0,0,0,1,0,0
4,5kYmeX,0,0,0,0,14,0,0,0,0,0,1112287852,0,0,0,0,0,0,0,1,0,0


In [33]:
# POSTULADO 1
# NO_POSTULADO 0
datos_random['postulado'] = 0
datos_random.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas,postulado
0,1Q4bd1R,1,0,1,0,2,0,1,0,0,0,1112397732,0,0,0,0,0,1,0,0,0,0,0
1,PmGkoMv,0,0,0,0,1,1,0,0,0,0,1112342096,0,0,0,0,1,0,0,0,0,0,0
2,ZDr5jaZ,0,0,0,0,1,0,0,0,0,0,1112225199,0,0,0,0,0,0,0,0,0,1,0
3,KBdV1oj,0,3,0,0,5,1,0,0,1,9,1112322633,0,0,0,0,0,0,0,1,0,0,0
4,5kYmeX,0,0,0,0,14,0,0,0,0,0,1112287852,0,0,0,0,0,0,0,1,0,0,0


In [34]:
# Estos ultimos son los datos no postulados
datos.drop_duplicates(subset=['idpostulante', 'idaviso'],inplace=True)

# 5. Datos de entrenamiento para el algoritmo

In [35]:
df = pd.concat([datos,datos_random])
df.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas,postulado
0,0z5Dmrd,0,0,0,0,4,0,0,0,0,0,1112384041,0,0,0,0,1,0,0,0,0,0,1
1,0z5Dmrd,0,0,0,0,4,0,0,0,0,0,1112420060,0,0,0,0,1,0,0,0,0,0,1
2,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112292169,0,0,0,0,1,0,0,0,0,0,1
3,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112309728,0,0,0,0,1,0,0,0,0,0,1
4,0z5JW1r,0,0,0,0,6,0,0,0,0,2,1112257171,0,0,0,0,1,0,0,0,0,0,1


In [36]:
# X Datos
# y Categorias
X = df.drop(columns=['idpostulante', 'idaviso', 'postulado'])
X.head()

,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,6,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,6,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,6,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0


In [37]:
y = df['postulado']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)  
print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

# 6. Datos finales

In [ ]:
final100k = pd.read_csv("../input/finalfinal/test_final_100k.csv")
final100k.head()

In [ ]:
final100k.info()

In [ ]:
# Recolecto su informacion en los distintos dataFrames
# Agrego datos de usuario
final = pd.merge(final100k, usuarios, on='idpostulante', how='left')
final.fillna(0, inplace=True)
final.head()

In [ ]:
# agrego datos de avisos
final = pd.merge(final, avisos, on='idaviso', how='left')
final.fillna(0, inplace=True)
final.head()

In [ ]:
final.head()

In [ ]:
X_final = final.drop(columns=['id', 'idaviso', 'idpostulante']) 
X_final.head()

# 7. PREDICCION

In [ ]:
y_final = clf.predict(X_final)

In [ ]:
y_final

# 8. SUMMIT

In [ ]:
y_summit = pd.DataFrame(y_final)
y_summit.columns = ['sepostulo']
y_summit.head()

In [ ]:
y_summit.info()

In [ ]:
summit = y_summit.reset_index()
summit.head()

In [ ]:
summit.rename(columns={'index':'id'}, inplace=True)
summit.head()

In [ ]:
# Guardo los datos
summit.to_csv("summit03DecTree.csv", index=False)
